## ETL Pipeline


What it does...

Export

In [14]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# load datasets
messages = pd.read_csv('../messages.csv')
categories = pd.read_csv('../categories.csv')

Transform

In [16]:
# merge datasets
df = messages.merge(categories, on='id')

In [17]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';', expand=True)

# fetch categories' names from the first row of data (it looks like: '[category_name]-[bool]')
row = categories.iloc[0,].str.split('-').apply(lambda x: x[0])
category_colnames = row.values
categories.columns = category_colnames

print(category_colnames)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


In [18]:
# replace categories' dummy string values (e.g.: '[category_name]-[bool]') with boolean values as int
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)

df.drop(columns=['categories'], inplace=True)

In [19]:
# concatenate messages w/ category features
df = pd.concat([df, categories], axis=1)

In [20]:
# drop duplicates
df.drop_duplicates(inplace=True)

Load

In [24]:
# load files to sql db
engine = create_engine('sqlite:///DB/disaster_messages.db')
df.to_sql('DB/disaster_messages', engine, index=False)

26216